In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
import torch.nn.functional as F


# Define your CNN model
class CNN_Model(nn.Module):
    def __init__(self, dropout_rate):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512 * 2 * 2, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Set the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.00001, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.3, 0.5, 0.7],
}

# Define transforms for train and validation data
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to a fixed size
    transforms.ToTensor(),
])

# Load train and val datasets from respective folders
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Define function for training and validation
def train_and_evaluate(params):
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    dropout_rate = params['dropout_rate']

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = CNN_Model(dropout_rate=dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training
    model.train()
    for epoch in range(5):  # Just a few epochs for hyperparameter search
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Perform hyperparameter tuning
best_params, best_accuracy = None, 0
for params in ParameterGrid(param_grid):
    accuracy = train_and_evaluate(params)
    print(f"Params: {params}, Accuracy: {accuracy:.2f}%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"Best Params: {best_params}, Best Accuracy: {best_accuracy:.2f}%")


Params: {'batch_size': 32, 'dropout_rate': 0.3, 'learning_rate': 0.001}, Accuracy: 42.86%
Params: {'batch_size': 32, 'dropout_rate': 0.3, 'learning_rate': 0.0001}, Accuracy: 42.86%
Params: {'batch_size': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001}, Accuracy: 57.14%
Params: {'batch_size': 32, 'dropout_rate': 0.5, 'learning_rate': 0.0001}, Accuracy: 42.86%
Params: {'batch_size': 64, 'dropout_rate': 0.3, 'learning_rate': 0.001}, Accuracy: 42.86%
Params: {'batch_size': 64, 'dropout_rate': 0.3, 'learning_rate': 0.0001}, Accuracy: 42.86%
Params: {'batch_size': 64, 'dropout_rate': 0.5, 'learning_rate': 0.001}, Accuracy: 42.86%
Params: {'batch_size': 64, 'dropout_rate': 0.5, 'learning_rate': 0.0001}, Accuracy: 42.86%
Best Params: {'batch_size': 32, 'dropout_rate': 0.5, 'learning_rate': 0.001}, Best Accuracy: 57.14%
